# reaction-network (Demo Notebook): Networks

### Author: Matthew McDermott
Last Updated: 12/12/21

The code provided in this notebook is an updated walkthrough of the first example (YMnO3) in the accompanying manuscript (see citation below). The refactored `reaction-network` package contains similar code to what was released with the manuscript; however, many processes/functions are now separated into their own defined classes/methods. For a look at the previous demo notebook (which also contained some of the raw results that went into the manuscript), please check out the _archived_ folder.

**If you use this code or Python package in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

### Imports

In [1]:
import logging 

from pymatgen.ext.matproj import MPRester
from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from rxn_network.thermo.chempot_diagram import ChemicalPotentialDiagram

from rxn_network.costs.softplus import Softplus
from pymatgen.core.composition import Composition, Element
from rxn_network.entries.entry_set import GibbsEntrySet
from rxn_network.network.network import ReactionNetwork
from rxn_network.entries.nist import NISTReferenceEntry
from rxn_network.reactions.computed import ComputedReaction
from rxn_network.reactions.reaction_set import ReactionSet
from rxn_network.reactions.open import OpenComputedReaction
from rxn_network.network.entry import NetworkEntry, NetworkEntryType
from rxn_network.network.visualize import plot_network_on_graphistry, plot_network
from rxn_network.pathways.solver import PathwaySolver

#import graphistry

%load_ext autoreload
%autoreload 2

logging.info("Logging initialized")

### Case Study: YMnO3 assisted metathesis

We will be using the assisted metathesis synthesis of YMnO3 as a case study for the reaction network code. This is the first example discussed in the original manuscript. The assisted metathesis reaction reported by Todd & Neilson (JACS, 2019) corresponds to a net reaction equation:

$$ Mn_2O_3 + 2 YCl_3 + 3Li_2CO_3 \to 2YMnO_3 + 6LiCl + 3CO_2 $$

In the paper, they report a reaction pathway involving the formation of intermediates LiMnO2 and YOCl. These react to form YMnO3 product and LiCl byproduct. (The CO2 is released when Li2CO3 reacts initially to form LiMnO2).

### Downloading and modifying entries

First, we acquire entries for phases in the Y-Mn-O-Li-Cl-C chemical system from the Materials Project (MP), a computed materials database containing calculations for over 130,000 materials.

In [2]:
with MPRester() as mpr:  # insert your Materials Project API key here if it's not stored in .pmgrc.yaml
    entries = mpr.get_entries_in_chemsys("Bi-O-Fe-K-F", inc_structure="final")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 957/957 [00:05<00:00, 184.45it/s]


The `GibbsEntrySet` class allows us to automatically converet `ComputedStructureEntry` objects downloaded from the MP database into `GibbsComputedEntry` objects, where DFT-calculated energies have been converted to machine learning (ML)-estimated equivalent values of the Gibbs free energies of formation, $\Delta G_f$ for all entries at the specified temperature. 

For more information, check out the citation in the documentation for `GibbsComputedEntry`.

In [3]:
temp = 900  # units: Kelvin
entry_set = GibbsEntrySet.from_entries(entries, temp, include_freed_data=True)

We can print the entries by calling `.entries` or `.entries_list`:

In [4]:
entry_set.entries

{FREEDReferenceEntry | Bi2O3
 Gibbs Energy (900 K) = -3.3635,
 FREEDReferenceEntry | BiF3
 Gibbs Energy (900 K) = -7.2191,
 FREEDReferenceEntry | BiO
 Gibbs Energy (900 K) = -0.2136,
 FREEDReferenceEntry | Fe2O3
 Gibbs Energy (900 K) = -6.0690,
 FREEDReferenceEntry | Fe3O4
 Gibbs Energy (900 K) = -8.5183,
 FREEDReferenceEntry | FeF2
 Gibbs Energy (900 K) = -6.0261,
 FREEDReferenceEntry | FeF3
 Gibbs Energy (900 K) = -8.6737,
 FREEDReferenceEntry | FeO
 Gibbs Energy (900 K) = -2.2090,
 FREEDReferenceEntry | K2O
 Gibbs Energy (900 K) = -2.2443,
 FREEDReferenceEntry | K2O2
 Gibbs Energy (900 K) = 0.3126,
 FREEDReferenceEntry | KF
 Gibbs Energy (900 K) = -4.9317,
 FREEDReferenceEntry | OF2
 Gibbs Energy (900 K) = 0.7947,
 GibbsComputedEntry | mp-1022722 | Bi3 O2 (Bi3O2)
 Gibbs Energy (900 K) = -2.6462,
 GibbsComputedEntry | mp-1062652 | Fe1 O2 (FeO2)
 Gibbs Energy (900 K) = -1.4627,
 GibbsComputedEntry | mp-1069079 | Fe2 Bi2 O6 (FeBiO3)
 Gibbs Energy (900 K) = -11.7059,
 GibbsComputedEntry

The `GibbsEntrySet` class has many helpful functions, such as the following `filter_by_stability()` function, which automatically removes entries which are a specified energy per atom above the convex hull of stability:

In [6]:
entry_set = entry_set.filter_by_stability(0.025)
entry_set.entries_list

[GibbsComputedEntry | mp-23152 | Bi2 (Bi)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-32597 | Bi13 O20 (Bi13O20)
 Gibbs Energy (900 K) = -34.7901,
 GibbsComputedEntry | mp-766354 | Bi25 O38 (Bi25O38)
 Gibbs Energy (900 K) = -66.6518,
 GibbsComputedEntry | mp-753309 | Bi4 O2 F8 (Bi2OF4)
 Gibbs Energy (900 K) = -27.1079,
 GibbsComputedEntry | mp-759405 | Bi12 O8 F20 (Bi3O2F5)
 Gibbs Energy (900 K) = -74.2636,
 GibbsComputedEntry | mp-757162 | Bi6 O8 F2 (Bi3O4F)
 Gibbs Energy (900 K) = -20.7596,
 GibbsComputedEntry | mp-30303 | Bi8 O14 (Bi4O7)
 Gibbs Energy (900 K) = -22.1389,
 GibbsComputedEntry | mp-1190898 | Bi7 O5 F11 (Bi7O5F11)
 Gibbs Energy (900 K) = -42.2368,
 FREEDReferenceEntry | BiF3
 Gibbs Energy (900 K) = -7.2191,
 GibbsComputedEntry | mvc-15980 | Bi2 F8 (BiF4)
 Gibbs Energy (900 K) = -17.1605,
 GibbsComputedEntry | mvc-3518 | Bi4 F20 (BiF5)
 Gibbs Energy (900 K) = -36.9774,
 GibbsComputedEntry | mp-762304 | Bi4 O4 F4 (BiOF)
 Gibbs Energy (900 K) = -19.7185,
 Gibb

In [25]:
[e.composition.reduced_formula for e in entry_set.entries_list]

['Bi',
 'Bi13O20',
 'Bi25O38',
 'Bi2OF4',
 'Bi3O2F5',
 'Bi3O4F',
 'Bi4O7',
 'Bi7O5F11',
 'BiF3',
 'BiF4',
 'BiF5',
 'BiOF',
 'BiOF3',
 'F2',
 'Fe',
 'Fe(Bi5O8)5',
 'Fe10OF19',
 'Fe2O3',
 'Fe3O4',
 'Fe4Bi2O9',
 'FeBi25O39',
 'FeBiO3',
 'FeF2',
 'FeF3',
 'FeO',
 'K',
 'K17Fe5O16',
 'K2BiF5',
 'K2FeF5',
 'K2O',
 'K2O2',
 'K3Bi',
 'K3BiO3',
 'K3BiO4',
 'K3Fe5F15',
 'K3FeF6',
 'K3FeO4',
 'K4Bi2O5',
 'K5Bi4',
 'K5FeO4',
 'K9Bi5O13',
 'KBi',
 'KBi2',
 'KBi3F10',
 'KBiF6',
 'KBiO2',
 'KBiO3',
 'KF',
 'KFeF4',
 'KFeO2',
 'KO2',
 'O2']

In this case, we remove all non-stable entries from the entry set, which greatly reduces the combinatorial complexity of the system:

## Building the reaction network

The reaction network can be initialized by providing 3 arguments to the `ReactionNetwork` class:

1. **entries:** iterable of entry-like objects (e.g., `GibbsComputedEntry`)
2. **enumerators:** iterable of enumerators which will be called during the build of the network
3. **cost_function:** the function used to calculate the cost of each reaction edge 

We will use a BasicEnumerator (see the **Enumerators Demo Notebook** for more information on the type of enumerators available):

In [8]:
be = BasicEnumerator()

The cost function is a monotonic function used to assign weights to edges in the network. In this case, we will use the softplus function, assigned a temperature scaling of $T=900$ K, and use the default arguments which automatically determine the softplus weighting based on the energy per atom of the reaction:

In [9]:
cf = Softplus(900)

Finally, we provide these as arugments to the `ReactionNetwork` initialization:

In [10]:
rn = ReactionNetwork(entry_set, [be], cf)

This simply initializes a `ReactionNetwork` object but does not build the network graph. To do so, we call the `.build()` function:

In [11]:
rn.build()

Ray is not initialized. Trying with environment variables!
Could not identify existing Ray instance. Creating a new one...


2022-03-14 17:00:36,919	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265


[{'NodeID': '8b7888ecf9b3bf9fe5fdf2b1374afccd0091b0180d649a69979c5d26', 'Alive': True, 'NodeManagerAddress': '127.0.0.1', 'NodeManagerHostname': 'MacBook-Pro-3.dhcp.lbl.gov.dhcp.lbl.gov', 'NodeManagerPort': 56338, 'ObjectManagerPort': 56337, 'ObjectStoreSocketName': '/tmp/ray/session_2022-03-14_17-00-33_089231_74555/sockets/plasma_store', 'RayletSocketName': '/tmp/ray/session_2022-03-14_17-00-33_089231_74555/sockets/raylet', 'MetricsExportPort': 57898, 'alive': True, 'Resources': {'object_store_memory': 4807687372.0, 'memory': 9615374747.0, 'node:127.0.0.1': 1.0, 'CPU': 12.0}}]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [01:08<00:00,  2.64s/it]
INFO:ReactionNetwork:Building graph from reactions...


This should have completed within a few seconds. You'll notice that two things happened:

1. The enumerator(s) were run and a list of reactions was generated
2. The weighted graph object was built with these reactions and stored under the `graph` attribute of the reaction network object

We can access this graph object, which is a graph-tool object, by using the `graph` attribute:

In [12]:
rn.graph

<Graph object, directed, with 2002 vertices and 12305 edges, 2 internal vertex properties, 3 internal edge properties, at 0x1496b41f0>

There are a couple provided ways to plot reaction networks. The first is to use the built in drawing features in graph-tool, which have been provided in a wrapper function:

In [13]:
plot_network(rn.graph);

KeyboardInterrupt: 

You'll notice that at this stage, the reaction network graph is a collection of "sub"-networks, i.e. a collection of smaller reaction networks for smaller chemical subsystems. This configuration will change once we set up for pathfinding in the next section.

The second way to plot graphs is to use graphistry, which requires setting up an account on Graphistry Hub: https://hub.graphistry.com/

In [14]:
plot_network_on_graphistry(rn.graph)

Must install optional dependencies: pygraphistry, networkx, and pyintergraph!


### Solving for reaction pathways

To solve for reaction pathways, we must set precursor phases, as well as a target phase. This will automatically build all the required "zero-cost" edges which connect the different chemical subsystems. Please see the original manuscript for more detail with regards to how this works. In short, zero-cost edges are drawn between and product node to any reactant node that contains a subset of the set consisting of the {precursors + products} phases. 

In [14]:
rn.set_precursors([entry_set.get_min_entry_by_formula("BiOF"), 
                   entry_set.get_min_entry_by_formula("KFeO2")])

In [15]:
rn.set_target("BiFeO3")

We can see how this changes the network by re-drawing it:

In [18]:
plot_network(rn.graph, output="Bi-Fe-O-K-F.png");

You should now see that the chemical subsystems have come together -- this is due to the zero-cost edges that were just described. We can now perform pathfinding to extract reaction pathways.

To get reaction pathways, we simply call the `find_pathways()` method. This automatically handles finding pathways to multiple targets, by calling the internal shortest paths method. The _k_ parameter specifies the number of shortest paths to find to each target.

In [35]:
paths = rn.find_pathways(["BiFeO3","KF"], k=3)

PATHS to FeBiO3 

--------------------------------------- 

BiOF + KFeO2 -> KF + FeBiO3 (dG = -0.138 eV/atom) 
Total Cost: 0.234 

6 BiOF -> Bi3O4F + Bi3O2F5 (dG = 0.035 eV/atom) 
2 KFeO2 + Bi3O2F5 -> K2BiF5 + 2 FeBiO3 (dG = -0.144 eV/atom) 
Total Cost: 0.506 

4 BiOF -> Bi3O4F + BiF3 (dG = 0.177 eV/atom) 
KFeO2 + 0.6667 BiF3 -> 0.3333 K3FeF6 + 0.6667 FeBiO3 (dG = -0.326 eV/atom) 
Total Cost: 0.507 

PATHS to KF 

--------------------------------------- 

BiOF + KFeO2 -> KF + FeBiO3 (dG = -0.138 eV/atom) 
Total Cost: 0.234 

2 KFeO2 -> Fe2O3 + K2O (dG = 0.18 eV/atom) 
BiOF + K2O -> KBiO2 + KF (dG = -0.42 eV/atom) 
Total Cost: 0.492 

BiOF + 0.1667 KFeO2 -> 0.1667 KFeF4 + 0.3333 Bi3O4F (dG = -0.006 eV/atom) 
1.5 KFeO2 + 0.5 KFeF4 -> Fe2O3 + 2 KF (dG = -0.168 eV/atom) 
Total Cost: 0.492 



The output of this method is a list of `BasicPathway` objects. Note that these objects contain a list of reactions and associated costs, but the actual pathway is typically not balanced:

In [36]:
print(paths[5])

BiOF + 0.1667 KFeO2 -> 0.1667 KFeF4 + 0.3333 Bi3O4F (dG = -0.006 eV/atom) 
1.5 KFeO2 + 0.5 KFeF4 -> Fe2O3 + 2 KF (dG = -0.168 eV/atom) 
Total Cost: 0.492


This means that the reactions you see above do not necessarily include all reactants, nor do they include form all desired products. They are simply a series of reactions extracted from the reaction network that maybe encountered as the system attempts to get from starter phases to target phases.

To actually get balanced reactions, we can use the `PathwaySolver` class. This class takes a set of entries, a list of `BasicPathway` objects, as well as a cost function, and can be used to solve for balanced pathways given a net reaction. First we initialize the class:

In [37]:
rn.entries.entries_list

[GibbsComputedEntry | mp-23152 | Bi2 (Bi)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-32597 | Bi13 O20 (Bi13O20)
 Gibbs Energy (900 K) = -34.7901,
 GibbsComputedEntry | mp-766354 | Bi25 O38 (Bi25O38)
 Gibbs Energy (900 K) = -66.6518,
 GibbsComputedEntry | mp-753309 | Bi4 O2 F8 (Bi2OF4)
 Gibbs Energy (900 K) = -27.1079,
 GibbsComputedEntry | mp-759405 | Bi12 O8 F20 (Bi3O2F5)
 Gibbs Energy (900 K) = -74.2636,
 GibbsComputedEntry | mp-757162 | Bi6 O8 F2 (Bi3O4F)
 Gibbs Energy (900 K) = -20.7596,
 GibbsComputedEntry | mp-30303 | Bi8 O14 (Bi4O7)
 Gibbs Energy (900 K) = -22.1389,
 GibbsComputedEntry | mp-1190898 | Bi7 O5 F11 (Bi7O5F11)
 Gibbs Energy (900 K) = -42.2368,
 FREEDReferenceEntry | BiF3
 Gibbs Energy (900 K) = -7.2191,
 GibbsComputedEntry | mvc-15980 | Bi2 F8 (BiF4)
 Gibbs Energy (900 K) = -17.1605,
 GibbsComputedEntry | mvc-3518 | Bi4 F20 (BiF5)
 Gibbs Energy (900 K) = -36.9774,
 GibbsComputedEntry | mp-762304 | Bi4 O4 F4 (BiOF)
 Gibbs Energy (900 K) = -19.7185,
 Gibb

In [38]:
ps = PathwaySolver(rn.entries, paths, Softplus(900)) # open_elem="O", chempot=0

To balance the pathways, we must provide a net reaction representing the total conversion of precursors to final products. This corresponds to the assisted metathesis reaction we defined in the beginning. We can automatically make this reaction by initializing a `ComputedReaction` object from the corresponding entries:

In [39]:
product_entries = []
for i in ["BiFeO3","KF"]:
    product_entries.append(entry_set.get_min_entry_by_formula(i))
    
net_rxn = ComputedReaction.balance(rn.precursors,product_entries)
net_rxn

BiOF + KFeO2 -> FeBiO3 + KF

Finally, we provide the net reaction to the `PathwaySolver` object. Note that the _intermediate_rxn_energy_cutoff_ helps to limit which intermediate reactions are considered (this can substantially decrease the combinatorial complexity), and the _filter_interdependent_ flag verifies that suggested pathways do not contain interdependnet reactions (i.e. where both the reactants of reaction A depend on the products of the reaction B, and the reactants of reaction B depend on the products of reaction A).

**Note: Even though this step is compiled/parallelized using Numba, this is often the most time-intensive step in the reaction network analysis. Consider limiting the value of the maximum number of combos, as well as the value of the intermediate reaction energy cutoff.**

In [41]:
balanced_paths = ps.solve(net_rxn, max_num_combos=5, 
                          intermediate_rxn_energy_cutoff=-0.01, 
                          use_minimize_enumerator=True,
                          filter_interdependent=True)

INFO:PathwaySolver:NET RXN: BiOF + KFeO2 -> FeBiO3 + KF 

INFO:PathwaySolver:Identifying reactions between intermediates...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 25.60it/s]
INFO:PathwaySolver:Found 76 intermediate reactions!
INFO:PathwaySolver:Solving for balanced pathways of size 4 ...
0/500000: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.18s/it]
INFO:PathwaySolver:Solving for balanced pathways of size 5 ...
4000000/4500000: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [03:01<00:00, 20.22s/it]


We can now print the suggested, balanced reaction pathways:

In [45]:
for idx, path in enumerate(balanced_paths):
    print(f"Path {idx+1}", "\n")
    print(path)
    print("\n")

In [43]:
from rxn_network.pathways.pathway_set import PathwaySet

In [44]:
path_set = PathwaySet.from_paths(balanced_paths)
path_set.get_paths()

We note that **Pathway 12 most closely matches the experimentally observed reaction pathway** (ordering subject to change in the future).

However, many of the pathways include hypothetical (never-before-synthesized) materials (e.g., Li3MnO3), so the top-ranked pathway does not necessarily match what is experimentally observed.

### Running networks with Fireworks

The `NetworkFW` class allows you to easily run the reaction network construction and pathfinding analysis via fireworks. Simply create a network firework, add it to the LaunchPad, and launch it on your computing resource. See the documentation for more information about each of the parameters.

In [70]:
from fireworks import LaunchPad, Workflow
from rxn_network.fireworks.core import NetworkFW

In [71]:
lpad = LaunchPad.auto_load()

In [72]:
fw = NetworkFW([BasicEnumerator()], Softplus(900), chemsys="Y-Mn-O-Li-Cl-C", entry_set_params={"e_above_hull":0.020}, 
               pathway_params={"precursors":["YCl3","Mn2O3","Li2CO3"], "targets":["YMnO3","LiCl","CO2"], "k":5},
              solver_params={"max_num_combos":5, "intermediate_rxn_energy_cutoff":0.0, "use_minimize_enumerator": True})

In [73]:
lpad.add_wf(Workflow([fw]))

2021-12-15 10:35:24,318 INFO Added a workflow. id_map: {-6: 3873}


INFO:launchpad:Added a workflow. id_map: {-6: 3873}


{-6: 3873}

In [74]:
!rlaunch singleshot

2021-12-15 10:35:28,591 INFO Hostname/IP lookup (this will take a few seconds)
2021-12-15 10:35:28,618 INFO Launching Rocket
2021-12-15 10:35:32,880 INFO RUNNING fw_id: 3873 in directory: /Users/mcdermott/PycharmProjects/reaction-network/notebooks
2021-12-15 10:35:33,031 INFO Task started: {{rxn_network.firetasks.build_inputs.EntriesFromDb}}.
2021-12-15 10:35:45,587 INFO Task completed: {{rxn_network.firetasks.build_inputs.EntriesFromDb}} 
2021-12-15 10:35:45,727 INFO Task started: {{rxn_network.firetasks.run_calc.BuildNetwork}}.
Li-Mn: 100%|████████████████████████████████████| 57/57 [10:02<00:00, 10.57s/it]
PATHS to YMnO3 

--------------------------------------- 

0.6667 Mn2O3 + Li2CO3 -> LiMnCO4 + 0.3333 Li3MnO3 (dG = 0.1 eV/atom) 
YCl3 + Li3MnO3 -> 3 LiCl + YMnO3 (dG = -0.215 eV/atom) 
Total Cost: 0.508 

Mn2O3 + 2 Li2CO3 -> LiMn(CO3)2 + Li3MnO3 (dG = 0.148 eV/atom) 
YCl3 + Li3MnO3 -> 3 LiCl + YMnO3 (dG = -0.215 eV/atom) 
Total Cost: 0.52 

0.3333 Mn2O3 + Li2CO3 -> CO2 + 0.6667 Li

### Thank you!

If any errors with the reaction-network code are encountered, please raise an Issue here: https://github.com/GENESIS-EFRC/reaction-network/issues

In [75]:
from maggma.stores import MongoStore
from rxn_network.pathways.pathway_set import PathwaySet

In [76]:
ms = MongoStore.from_db_file("/Users/mcdermott/db_rn.json")
ms.connect()

In [77]:
d = ms.query_one({"task_id":82}, ["balanced_pathways"])

In [78]:
pset = PathwaySet.from_dict(d["balanced_pathways"])